In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow
import keras
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop, Adam
from sklearn.model_selection import train_test_split
from keras.callbacks import ReduceLROnPlateau



# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
#Load data
df_train = pd.read_csv("../input/train.csv")
df_test = pd.read_csv("../input/test.csv")

In [ ]:
df_train_without_labels = df_train.drop(labels='label', axis=1)
df_train_labels = df_train['label']


#train and test set preparation
x_train, x_test, y_train, y_test = train_test_split(df_train_without_labels.values, df_train_labels.values, test_size=0.2, random_state=42)
print('x_train shape : ',x_train.shape)
print('x_test shape : ',x_test.shape)

x_train = x_train/255
x_test = x_test/255

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

#sample of data
plt.imshow(x_train[1].reshape(28,28), cmap = plt.cm.binary)
plt.show()

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(64, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation = 'softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.0001),
              metrics=['accuracy'])



In [ ]:
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
history = model.fit(x_train, y_train, epochs=30,validation_split=0.05, batch_size= 128,callbacks=[learning_rate_reduction])
history.history.keys()

In [ ]:
val_loss, val_acc = model.evaluate(x_test, y_test)
print(val_loss, val_acc)

In [ ]:
predictions = model.predict_classes(df_test)


In [ ]:
submission = pd.read_csv("../input/sample_submission.csv")
submission['Label']=predictions

submission.head()
submission.to_csv("keras.csv", index=False, header=True)
